In [ ]:
import pandas as pd         # 导入pandas库，它就像一个超级强大的Excel工具，专门用来处理表格数据（DataFrame）。
from sklearn.model_selection import train_test_split # 从sklearn.model_selection模块导入train_test_split函数，用于将数据集划分为训练集和测试集。
from sklearn.linear_model import LinearRegression # 从sklearn.linear_model模块导入LinearRegression类，这是我们将要使用的线性回归模型。
from sklearn.preprocessing import StandardScaler # 从sklearn.preprocessing模块导入StandardScaler类，用于对数据进行标准化（特征缩放）。
from sklearn.pipeline import Pipeline # 从sklearn.pipeline模块导入Pipeline类，用于将多个数据处理步骤和模型组合成一个单一的对象。
import pickle               # 导入pickle库，用于序列化和反序列化Python对象，这里主要用于保存和加载训练好的模型。
from sklearn.ensemble import RandomForestRegressor # 从sklearn.ensemble模块导入RandomForestRegressor类，这是我们将要使用的随机森林回归模型。

# 加载数据集
df = pd.read_csv('auto-mpg.csv') # 使用pandas的read_csv函数，读取名为 'auto-mpg.csv' 的CSV文件到DataFrame `df` 中。

# 显示前五行数据
print(df.head())            # 打印DataFrame `df` 的前5行，帮助用户快速预览数据。

# 处理缺失值
# 将 'horsepower' 列中的所有值转换为数值类型
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')
# `df['horsepower']` 选取 'horsepower' 列。
# `pd.to_numeric(...)` 尝试将这一列转换为数值类型。
# `errors='coerce'` 是一个重要的参数：如果遇到无法转换为数字的值（例如，文本字符串'?'），它不会报错，而是将其替换为 `NaN`（缺失值）。
# 删除包含缺失值的行
df = df.dropna()            # 使用DataFrame的 `dropna()` 方法，删除所有包含任何缺失值（NaN）的行。
                            # 由于上一步可能产生了新的 `NaN` 值，这里用于清理这些不合规的记录。

# 选择相关特征进行建模（定义自变量（返回一个DataFrame）和因变量）
X = df.drop(columns=['mpg','car name'])
# 从DataFrame `df` 中删除 'mpg'（目标变量）和 'car name'（汽车名称，通常不直接用于数值回归模型）。
# `drop(columns=...)` 明确指定删除列。剩余的列作为特征（自变量），赋值给 `X`。
y = df['mpg']               # 选取 'mpg' 列作为目标变量（因变量），赋值给 `y`。

# 将数据集划分为训练集和测试集（测试集占比20%）
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
# 使用train_test_split函数将数据集划分为训练集和测试集。
# `X`, `y`: 要分割的特征和目标变量。
# `test_size=0.2`: 测试集占总数据的20%。
# `random_state=42`: 随机种子，确保每次运行代码时分割结果一致。

# 创建包含标准化和线性回归的管道
pipeline = Pipeline([('scaler', StandardScaler()),('linreg', LinearRegression())])
# 创建一个 `Pipeline`（管道）对象。管道可以方便地将多个处理步骤（如特征缩放）和模型（如线性回归）串联起来。
# `('scaler', StandardScaler())`: 管道的第一个步骤是特征标准化，使用StandardScaler。
# `('linreg', LinearRegression())`: 管道的第二个步骤是线性回归模型。
# 这样，当管道被训练时，数据会先经过标准化，然后才送入线性回归模型。当进行预测时，新数据也会自动先标准化再预测。

# 训练模型
pipeline.fit(X_train, y_train) # 使用训练集特征 `X_train` 和训练集目标 `y_train` 训练整个管道。
                               # 在此过程中，StandardScaler会先fit（学习）训练集的均值和标准差并transform数据，
                               # 然后LinearRegression会在转换后的数据上进行训练。

# 保存训练好的模型
with open('2.2.2_model.pkl', 'wb') as model_file: # 以二进制写入模式 ('wb') 打开一个文件，用于保存模型。
    pickle.dump(pipeline,model_file) # 使用pickle.dump() 将训练好的 `pipeline` 对象序列化并写入文件。

# 预测并保存结果
y_pred = pipeline.predict(X_test) # 使用训练好的 `pipeline` 对测试集特征 `X_test` 进行回归预测。
                                   # `X_test` 会自动先经过管道内的StandardScaler进行标准化，然后才由LinearRegression进行预测。
results_df = pd.DataFrame(y_pred, columns=['预测结果']) # 将预测结果 `y_pred` 转换为一个pandas DataFrame，列名为 '预测结果'。
results_df.to_csv('2.2.2_results.txt', index=False) # 将预测结果DataFrame保存为CSV文件 `2.2.2_results.txt`。
                                                    # `index=False` 表示在保存时不要写入DataFrame的索引。

# 测试模型
with open('2.2.2_report.txt', 'w') as results_file: # 以写入模式 ('w') 打开一个文本文件，用于保存报告。
    results_file.write(f'训练集得分: {pipeline.score(X_train, y_train)}\n')
    # `pipeline.score(X_train, y_train)` 计算模型在训练集上的性能得分。
    # 对于回归模型，这个得分通常是R²（决定系数），表示模型解释因变量方差的比例。
    results_file.write(f'测试集得分: {pipeline.score(X_test, y_test)}\n')
    # `pipeline.score(X_test, y_test)` 计算模型在测试集上的性能得分 (R²)。
    # 测试集得分更能反映模型的泛化能力。

# 创建随机森林回归模型实例（创建的决策树的数量为100）
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
# 创建一个RandomForestRegressor模型实例。
# `n_estimators=100`: 指定森林中包含100棵决策树。
# `random_state=42`: 随机种子，确保模型训练结果的可复现性。
# 训练随机森林回归模型
rf_model.fit(X_train,y_train) # 使用训练集特征 `X_train` 和训练集目标 `y_train` 训练随机森林回归模型。
                               # 注意，这里随机森林模型没有在标准化数据上训练，因为决策树类模型对特征缩放不敏感。

# 使用随机森林模型进行预测
y_pred_rf = rf_model.predict(X_test) # 使用训练好的 `rf_model` 对测试集特征 `X_test` 进行回归预测。

# 保存新的结果
results_rf_df = pd.DataFrame(y_pred_rf, columns=['预测结果']) # 将随机森林模型的预测结果 `y_pred_rf` 转换为DataFrame。
results_rf_df.to_csv('2.2.2_results_rf.txt', index=False) # 将这个DataFrame保存为CSV文件 `2.2.2_results_rf.txt`。

# 测试模型并保存得分
with open('2.2.2_report_rf.txt', 'w') as results_rf_file: # 打开文件保存随机森林模型的报告。
    results_rf_file.write(f'训练集得分: {rf_model.score(X_train, y_train)}\n') # 写入随机森林模型在训练集上的R²得分。
    results_rf_file.write(f'测试集得分: {rf_model.score(X_test, y_test)}\n') # 写入随机森林模型在测试集上的R²得分。
